# Hybrid Recommendation

In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [ ]:
# Load the MovieLens dataset (you can download it from https://grouplens.org/datasets/movielens/)
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [ ]:
# Merge movie and rating data
movie_ratings = pd.merge(movies, ratings, on='movieId')

## Content-Based Recommendation

In [ ]:
# TF-IDF vectorization of movie genres
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(movies['genres'])

In [ ]:
# Calculate cosine similarity between movies based on genres
content_similarity = cosine_similarity(tfidf_matrix, tfidf_matrix)

## Collaborative Filtering Recommendation

In [ ]:
# Create a user-item matrix
user_item_matrix = movie_ratings.pivot_table(index='userId', columns='title', values='rating')

In [ ]:
# Fill missing values with 0
user_item_matrix = user_item_matrix.fillna(0)

In [ ]:
# Convert to sparse matrix
sparse_matrix = csr_matrix(user_item_matrix)

In [ ]:
def recommend_movies(movie_title):
    # Find the index of the movie in the dataset
    movie_idx = movies.index[movies['title'] == movie_title].tolist()[0]

In [ ]:
    # Calculate content-based and collaborative filtering scores
    content_scores = list(enumerate(content_similarity[movie_idx]))
    collab_scores = list(enumerate(knn_model.kneighbors([sparse_matrix[movie_idx]], n_neighbors=10)[1][0]))

In [ ]:
    # Sort by scores in descending order
    content_scores = sorted(content_scores, key=lambda x: x[1], reverse=True)
    collab_scores = sorted(collab_scores, key=lambda x: x[1])

In [ ]:
    # Get movie IDs from the sorted lists
    content_movie_indices = [i[0] for i in content_scores]
    collab_movie_indices = [i[0] for i in collab_scores]

In [ ]:
    # Recommend movies based on a combination of content-based and collaborative filtering
    recommended_movies = movies.iloc[collab_movie_indices[:5]]  # Get top 5 collaborative filtering recommendations
    recommended_movies = recommended_movies.append(movies.iloc[content_movie_indices[:5]])  # Add top 5 content-based recommendations
    return recommended_movies

In [ ]:
# Example usage:
movie_to_recommend = "Toy Story (1995)"
recommended_movies = recommend_movies(movie_to_recommend)
print("Recommended Movies for '{}':".format(movie_to_recommend))
print(recommended_movies[['title', 'genres']])